In [1]:
import numpy as np
from acv_explainers import ACXplainer
from acv_explainers.utils import *
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from utils import MyTabNetClassifier
from utils import DatasetHelper, DATASETS_NAME
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

if(model=='L'):
    print('* Classifier: LogisticRegression')
    mdl = LogisticRegression(penalty='l2', C=1.0, solver='liblinear')
    print('\t* C: {}'.format(mdl.C)); print('\t* penalty: {}'.format(mdl.penalty));
elif(model=='X'):
    print('* Classifier: LightGBM')
    mdl = LGBMClassifier(n_estimators=50, num_leaves=8)
    print('\t* n_estimators: {}'.format(mdl.n_estimators)); print('\t* num_leaves: {}'.format(mdl.num_leaves));
elif(model=='T'):
    print('* Classifier: TabNet')
    mdl = MyTabNetClassifier(D.feature_types, verbose=0)

seed = 2022    
dataset = 'c'
dataset_name = DATASETS_NAME[dataset]
model= 'X'
np.random.seed(0)

D = DatasetHelper(dataset=dataset, feature_prefix_index=False)
X_tr, X_ts, y_tr, y_ts = D.train_test_split()

isolation = IsolationForest()
isolation.fit(X_tr)

mdl = LGBMClassifier(n_estimators=50, num_leaves=8)
mdl.fit(X_tr, y_tr)

x_train = X_tr.copy()
x_test = X_ts.copy()

* Classifier: LightGBM
	* n_estimators: 50
	* num_leaves: 8


In [2]:
results = load_model('/home/xxxxxxx/Documents/experiments_counterfactualsrules2022/COMPASCR_results')

In [43]:
if np.mean(mdl.predict(results.x_test) == results.y_test):
    print('CONSISTENT')
else:
    raise ValueError


CONSISTENT


In [58]:
x = []
for i, c in enumerate(results.counterfactuals_samples_local):
    if len(c) !=0:
        x.append(results.x_test[i])

x = np.array(x)
ce = np.array(results.dist_local)
ce_r = np.array(results.dist_regional)

print('all acc', np.mean(mdl.predict(x_test) != mdl.predict(ce_r)))

all acc 0.9494163424124513


In [59]:
x_pos = x[mdl.predict(x) == 1]
ce_pos = ce[mdl.predict(x) == 1]

print('LOCAL positive accuracy', np.mean(mdl.predict(x_pos) != mdl.predict(ce_pos)))

LOCAL positive accuracy 0.9945945945945946


In [60]:
print('LOCAL positive sparsity', np.mean(np.sum(x_pos-ce_pos!=0, axis=1)))

LOCAL positive sparsity 2.2486486486486488


In [61]:
inlier_pos = np.mean(results.isolation.predict(ce_pos) == 1)
print('LOCAL positive inlier', inlier_pos)

LOCAL positive inlier 0.8756756756756757


In [62]:
x_neg = x[mdl.predict(x) == 0]
ce_neg = ce[mdl.predict(x) == 0]

print('LOCAL negative accuracy', np.mean(mdl.predict(x_neg) != mdl.predict(ce_neg)))

LOCAL negative accuracy 0.8451178451178452


In [63]:
print('LOCAL negative sparsity', np.mean(np.sum(x_neg-ce_neg!=0, axis=1)))

LOCAL negative sparsity 4.205387205387205


In [64]:
inlier_neg = np.mean(results.isolation.predict(ce_neg) == 1)
print('LOCAL negative inlier', inlier_neg)

LOCAL negative inlier 0.7373737373737373


In [37]:
x_pos_r = x_test[mdl.predict(x_test) == 1]
ce_pos_r = ce_r[mdl.predict(x_test) == 1]

print('REGIONAL positive accuracy', np.mean(mdl.predict(x_pos_r) != mdl.predict(ce_pos_r)))

positive accuracy 0.8977635782747604


In [38]:
print('REGIONAL positive sparsity', np.mean(np.sum(x_pos_r-ce_pos_r!=0, axis=1)))

positive sparsity 2.912140575079872


In [39]:
inlier_pos = np.mean(results.isolation.predict(ce_pos_r) == 1)
print('REGIONAL positive inlier', inlier_pos)

positive inlier 0.7428115015974441


In [40]:
x_neg_r = x_test[mdl.predict(x_test) == 0]
ce_neg_r = ce_r[mdl.predict(x_test) == 0]

print('REGIONAL negative accuracy', np.mean(mdl.predict(x_neg_r) != mdl.predict(ce_neg_r)))

negative accuracy 0.9847161572052402


In [41]:
print('REGIONAL negative sparsity', np.mean(np.sum(x_neg_r-ce_neg_r!=0, axis=1)))

negative sparsity 3.0731441048034935


In [42]:
inlier_neg_r = np.mean(results.isolation.predict(ce_neg_r) == 1)
print('REGIONAL negative inlier', inlier_neg_r)

negative inlier 0.9388646288209607


In [83]:
print('Local Coverage = {} -- Global Coverage {}'.format(results.coverage_local, 
                                                        results.coverage_regional))

Local Coverage = 0.964 -- Global Coverage 1.0
